In [1]:
#Import das bibliotecasx
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import math

In [2]:
# Criação do dicionário que receberá os registros
dic_produtos = {'produto':[], 'preco':[], 'desconto':[], 'link':[]}

# Define os headers que serão utilizados no programa
headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36"}

In [3]:
#FutFanatics - Parte 1
url = 'https://www.futfanatics.com.br/loja/catalogo.php?loja=311840&categoria=1161&variacao=tamanho_41&range=&order=2&pg=1'

site = requests.get(url, headers=headers)

encoding = site.encoding if 'charset' in site.headers.get('content-type', '').lower() else None
parser = 'html5lib'
soup = BeautifulSoup(site.content, parser, from_encoding=encoding)

totais = soup.find_all('div', class_="search-counter")

total_texto = totais[0].get_text().strip()

index = total_texto.find(' ')

qtd  = total_texto[:index]

ultima_pagina = math.ceil(int(qtd)/40)

print("Quantidade de produtos (FutFanatics):",qtd)
print("Quantidade de páginas (FutFanatics):",ultima_pagina)

Quantidade de produtos (FutFanatics): 585
Quantidade de páginas (FutFanatics): 15


In [4]:
#FutFanatics - Parte 2
for i in range(1, ultima_pagina+1):
    print('FutFanatics - página', i, 'de', ultima_pagina)
    url_pag = f'https://www.futfanatics.com.br/loja/catalogo.php?loja=311840&categoria=1161&variacao=tamanho_41&range=&order=2&pg={i}'
    site = requests.get(url_pag, headers=headers)
    soup = BeautifulSoup(site.content, parser, from_encoding=encoding)
    produtos = soup.find_all('div', class_=re.compile('product-item'))

    for produto in produtos:
        prod = produto.find('div', class_=re.compile('product-name')).get_text().strip()
        preco = produto.find('div', class_='price').get_text().strip()
        desconto = 0
        descontos = produto.find_all('span', class_=re.compile('discount'))
        if len(descontos) > 0:
            desconto = descontos[0].get_text().strip()
        link = produto.find_all('a', href=True)[0]['href']
        dic_produtos['produto'].append(prod)
        dic_produtos['preco'].append(preco)
        dic_produtos['desconto'].append(desconto)
        dic_produtos['link'].append(link)

FutFanatics - página 1 de 15
FutFanatics - página 2 de 15
FutFanatics - página 3 de 15
FutFanatics - página 4 de 15
FutFanatics - página 5 de 15
FutFanatics - página 6 de 15
FutFanatics - página 7 de 15
FutFanatics - página 8 de 15
FutFanatics - página 9 de 15
FutFanatics - página 10 de 15
FutFanatics - página 11 de 15
FutFanatics - página 12 de 15
FutFanatics - página 13 de 15
FutFanatics - página 14 de 15
FutFanatics - página 15 de 15


In [5]:
# Cria o DataFrame Pandas com os produtos capturados
df = pd.DataFrame(dic_produtos)
df.head()

,produto,preco,desconto,link
0,Chuteira Adidas Predator Accuracy 23.1 Low FG ...,"R$1.699,90",5%,https://www.futfanatics.com.br/chuteira-adidas...
1,Chuteira Adidas Predator Accuracy 23.1 FG Camp...,"R$1.599,90",11%,https://www.futfanatics.com.br/chuteira-adidas...
2,Chuteira Adidas Copa Pure 23.1 FG Campo Preta,"R$1.599,90",11%,https://www.futfanatics.com.br/chuteira-adidas...
3,Chuteira Puma Future Ultimate FG/AG Campo Bran...,"R$1.499,90",0,https://www.futfanatics.com.br/chuteira-puma-f...
4,Chuteira Mizuno Morelia Neo Campo Preta e Preta,"R$1.499,90",0,https://www.futfanatics.com.br/chuteira-mizuno...


In [6]:
# Faz algumas transformações nas colunas capturadas
tabela = df.copy()
tabela['preco'] = tabela['preco'].apply(lambda x: str(x).replace(".",'').replace("R$",''))
tabela['preco'] = pd.to_numeric(tabela['preco'].apply(lambda x: str(x).replace(",",'.')))
tabela['desconto'] = pd.to_numeric(tabela['desconto'].apply(lambda x: str(x).replace("%",'')))
tabela['marca'] = tabela['produto'].astype(str).str.split().str[1]
tabela = tabela.sort_values(by=['desconto'],ascending=False)
tabela.head(15)

,produto,preco,desconto,link,marca
16,Chuteira Umbro Tocco II Pro FG Campo Preta,659.9,56,https://www.futfanatics.com.br/chuteira-umbro-...,Umbro
581,Chuteira Oxn Velox 4 Neo Campo Amarela,69.9,53,https://www.futfanatics.com.br/chuteira-oxn-ve...,Oxn
582,Chuteira Oxn Gênio 3 Campo Preta,69.9,50,https://www.futfanatics.com.br/chuteira-oxn-ge...,Oxn
580,Chuteira Oxn Gênio 3 Campo Black,69.9,50,https://www.futfanatics.com.br/chuteira-oxn-ge...,Oxn
579,Chuteira Umbro Cosmic Campo Preta E Cinza,79.9,50,https://www.futfanatics.com.br/chuteira-umbro-...,Umbro
583,Chuteira Oxn Agilis 2 Campo Marinho,69.9,46,https://www.futfanatics.com.br/chuteira-oxn-ag...,Oxn
577,Chuteira Oxn Fusion Grip 3 Neo Campo Marinho,79.9,46,https://www.futfanatics.com.br/chuteira-oxn-fu...,Oxn
524,Chuteira Umbro Play Campo Preta,99.9,44,https://www.futfanatics.com.br/chuteira-umbro-...,Umbro
309,Chuteira Penalty RX Locker Y-3 Campo Azul e Verde,159.9,44,https://www.futfanatics.com.br/chuteira-penalt...,Penalty
538,Chuteira Umbro Ultraskin Futsal Amarelo,99.9,44,https://www.futfanatics.com.br/chuteira-umbro-...,Umbro


In [7]:
# Exporta o resultado para CSV
tabela.to_csv('precoChuteiras.csv', encoding='utf-8', sep=',', index=False)